In [1]:
from Preprocessing import *

Device available:  cuda


In [2]:
# Create a test dataset, run the trained model on it, and write predictions to CSV.
test_dir = BASE_DIR/'Dataset/cifar-10/test'
path_len = len(str(test_dir))+1
test_files = sorted([test_dir/f for f in os.listdir(test_dir)], key = lambda x: int(str(x)[path_len:-4]))

test_df = pd.DataFrame(test_files, columns=['path'])
test_df['label'] = [int(str(x)[path_len:-4]) for x in test_files]  # dummy labels for compatibility

test_dataset = CifarDataset(test_df, transform)  # use the previously defined `transform`
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

pred_ids = []
pred_labels = []

test_model = efficientnet_b0(weights='DEFAULT')
test_model.features[0][0].stride = (1,1)
test_model.classifier = nn.Sequential(
    nn.Dropout(p=0.3),
    nn.Linear(test_model.classifier[1].in_features, 10)
)
test_model=test_model.to(device)
test_model.load_state_dict(torch.load("efficientnetmodeltweaked.pth", map_location=device))


<All keys matched successfully>

In [3]:
test_model.eval()

with torch.no_grad():
    for imgs, id in test_loader:
        imgs = imgs.to(device)
        id = id.to(device)
        ensemble_outputs = torch.zeros((imgs.size(0), N_Models , 10))
        for i in range(N_Models):
            outputs = test_model(imgs)
            ensemble_outputs[:, i, :] = outputs
        avg_outputs = ensemble_outputs.mean(dim=1).to(device)
        preds = avg_outputs.argmax(dim=1).cpu().numpy()
        labels = le.inverse_transform(preds)
        pred_ids.extend(id.tolist())
        pred_labels.extend(labels.tolist())

submission_df = pd.DataFrame({"id": pred_ids, "label": pred_labels})
out_csv = "efficientnetmodeltweaked.csv"
submission_df.to_csv(out_csv, index=False)
print(f"Saved predictions to {out_csv}")

Saved predictions to efficientnetmodeltweaked.csv
